# Logistic Regression example

In [2]:
// cargo install the newest version
// this may take a while, needs to download and install the library
:dep smartcore = "0.3.0"

In [3]:
use smartcore::dataset::iris::load_dataset as iris_load;
use smartcore::dataset::breast_cancer::load_dataset as bc_load;
use smartcore::dataset::diabetes::load_dataset as db_load;
use smartcore::linear::logistic_regression::LogisticRegression;
use smartcore::linalg::basic::matrix::DenseMatrix;
// Model performance
use smartcore::metrics::accuracy;

use smartcore::model_selection::{cross_val_predict, cross_validate, KFold};

## basic prediction and accuracy

In [4]:
// Load Iris dataset
let iris_dataset = iris_load();

// Turn Iris dataset into NxM matrix
// Input data
let x: DenseMatrix<f32> = DenseMatrix::new(
    iris_dataset.num_samples,      // num rows
    iris_dataset.num_features,     // num columns
    iris_dataset.data,             // data as Vec
    false,                         // column_major
);
// These are our target class labels
let y: Vec<u32> = iris_dataset.target;

// Fit Logistic Regression to Iris dataset
let lr = LogisticRegression::fit(&x, &y, Default::default()).unwrap();
let y_hat = lr.predict(&x).unwrap(); // Predict class labels

// Calculate training error
println!("accuracy: {}", accuracy(&y, &y_hat)); // Prints 0.98

accuracy: 0.9866666666666667


## basic cross validation

In [5]:
// we need to import this as LogisticRegression is a SupervisedEstimator
// and we need its traits
use smartcore::api::SupervisedEstimator

// This example is expired by
// https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_predict.html
// Load dataset
let breast_cancer_data = bc_load();
let x: DenseMatrix<f32> = DenseMatrix::new(
    breast_cancer_data.num_samples,
    breast_cancer_data.num_features,
    breast_cancer_data.data,
    false
);
// These are our target values
let y = breast_cancer_data.target;
// cross-validated estimator
let results = cross_validate(
    LogisticRegression::new(),
    &x,
    &y,
    Default::default(),
    &KFold::default().with_n_splits(3),
    &accuracy,
)
.unwrap();
println!(
    "Test score: {}, training score: {}",
    results.mean_test_score(),
    results.mean_train_score()
);


Test score: 0.9561125034809246, training score: 0.9639864833587928


## basic cross validation with prediction

In [6]:
use smartcore::linear::linear_regression::LinearRegression;

// Load Diabetes dataset
let diabetes_data = db_load();
let x = DenseMatrix::new(
    diabetes_data.num_samples,
    diabetes_data.num_features,
    diabetes_data.data,
    false
);

// These are our target values
let y = diabetes_data.target;

// Generate cross-validated estimates for each input data point
let y_hat = cross_val_predict(
    LinearRegression::new(),
    &x,
    &y,
    Default::default(),
    &KFold::default().with_n_splits(10),
)
.unwrap();


println!(
    "y hat {:?}",
    &y_hat
);


y hat [202, 69, 173, 165, 128, 104, 72, 122, 161, 220, 93, 92, 116, 162, 97, 173, 217, 181, 148, 125, 124, 85, 115, 270, 165, 147, 95, 182, 126, 182, 156, 72, 249, 116, 79, 88, 208, 160, 237, 139, 158, 70, 150, 74, 223, 123, 143, 108, 75, 185, 157, 169, 135, 162, 139, 70, 211, 81, 95, 133, 116, 179, 58, 100, 115, 188, 146, 124, 113, 118, 72, 235, 133, 123, 152, 129, 192, 72, 164, 93, 177, 120, 61, 148, 54, 165, 35, 153, 84, 106, 75, 187, 196, 59, 108, 123, 208, 209, 124, 143, 165, 102, 149, 156, 150, 116, 75, 159, 226, 137, 33, 119, 150, 206, 297, 189, 212, 234, 163, 151, 158, 201, 219, 183, 168, 190, 56, 110, 93, 211, 245, 69, 114, 66, 145, 237, 58, 235, 252, 251, 157, 231, 172, 119, 181, 238, 188, 232, 111, 178, 205, 147, 197, 125, 150, 199, 148, 125, 87, 234, 83, 236, 149, 194, 143, 78, 54, 261, 221, 231, 50, 85, 218, 94, 164, 117, 156, 222, 100, 165, 178, 93, 176, 158, 202, 188, 199, 64, 153, 118, 194, 131, 88, 139, 156, 170, 101, 193, 139, 181, 93, 68, 166, 198, 176, 235, 162, 213

In [7]:
:dep plotters = { version = "^0.3.0", default_features = false, features = ["evcxr", "all_series"] }

use plotters::prelude::*;

//Assemble XY dataset for the scatter plot
let xy: Vec<Vec<u32>> = y.clone()
        .into_iter()
        .zip(y_hat.clone().into_iter())
        .map(|(x1, x2)| vec![x1, x2])
        .collect();

let diff: Vec<Vec<u32>> = y.clone()
        .into_iter()
        .zip(y_hat.clone().into_iter())
        .map(|(x1, x2)| vec![(u32::max(x1, x2) - u32::min(x1, x2)), x2])
        .collect();

// println!("{:?}",
//     &xy
// );

evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Y and Y_hat (green) and their difference (blue)", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0u32..350u32, 0u32..400u32)?;
    
    chart.configure_mesh()
        .x_desc("Y")
        .y_desc("Y_hat")
        .draw()?;
    
    chart.draw_series(xy.iter().map(|x| Circle::new((x[0], x[1]), 3, GREEN.filled())))?;
    
    chart.draw_series(diff.iter().map(|x| Circle::new((x[0], x[1]), 3, BLUE.filled())))?;
    
    // You can alawys freely draw on the drawing backend
    let area = chart.plotting_area();
    let sd = 0.13;
    let two_sigma = sd * 2.0;
    area.draw(&Rectangle::new(
        [((0.5 - two_sigma) as u32, (0.5 - two_sigma)  as u32),
            ((0.5 + two_sigma)  as u32, (0.5 + two_sigma)  as u32)], 
        RED.mix(0.3).filled())
    )?;
    
    Ok(())
}).style("width: 60%")

Y and Y_hat (green) and their difference (blue)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Y_hat
 
 
Y
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
350
 
 
 
400
 
 
 
 
0
 
 
 
50
 
 
 
100
 
 
 
150
 
 
 
200
 
 
 
250
 
 
 
300
 
 
 
350